In [95]:
#Importer vigtige pakker
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
import pandas_datareader.data as web
import datetime
import pydst
dst = pydst.Dst(lang='en')

In [96]:
# a. loading data
dst.get_variables('NABP10') # command to se 'id'
dst.get_variables('NABP10').iloc[0]['values'] # command to se 'values' in 'TRANSAKT'

# Vi henter kun nominelle priser, da "Aflønning af ansatte" ikke er angivet i faste priser 
gva10 = dst.get_data(table_id = 'NABP10', variables={'TRANSAKT':['B1GD','D1D'],'BRANCHE':['*'], 'PRISENHED':['V'], 'TID':['*']})
emp10 = dst.get_data(table_id = 'NABB10', variables={'SOCIO':['EMPH_DC','SALH_DC'],'BRANCHE':['*'], 'TID':['*']})

In [135]:
# b. define dictionaries to rename variables
columns_dict = {}
columns_dict['TRANSAKT'] = 'variable'
columns_dict['BRANCHE'] = 'industry'
columns_dict['PRISENHED'] = 'unit'
columns_dict['TID'] = 'year'
columns_dict['INDHOLD'] = 'value'
columns_dict['SOCIO'] = 'employment'

var_dict = {} # var is for variable
var_dict['B.1g Gross value added'] = 'GVA'
var_dict['D.1 Compensation of employees'] = 'ComEmp'
var_dict['Hours worked (1,000 hours)'] = 'Hours_TOT'
var_dict['Hours worked for employees (1,000 hours)'] = 'Hours_Emp'

unit_dict = {}
unit_dict['2010-prices, chained values'] = 'real'
unit_dict['Current prices'] = 'nominal'

# b. i
gva10.rename(columns=columns_dict,inplace=True)
emp10.rename(columns=columns_dict,inplace=True)

for key,value in var_dict.items():
    gva10.variable.replace(key,value, inplace=True)
    emp10.employment.replace(key,value, inplace=True)

for key,value in unit_dict.items():
    gva10.unit.replace(key,value, inplace=True)

# b. ii redefine value from 'str' to 'float'
emp10.value = emp10.value.astype('float')
gva10['value'] = gva10['value'].apply(pd.to_numeric, errors='coerce')



,variable,industry,unit,year,value
0,GVA,Total,nominal,1966,73555
1,GVA,Total,nominal,1967,80538
2,GVA,Total,nominal,1968,88782
3,GVA,Total,nominal,1969,101289
4,GVA,Total,nominal,1970,112399
5,GVA,Total,nominal,1971,125050
6,GVA,Total,nominal,1972,142119
7,GVA,Total,nominal,1973,165043
8,GVA,Total,nominal,1974,185957
9,GVA,Total,nominal,1975,208105


In [126]:
# Split dataframe emp10 in two - hours total and hours for employees.
H_TOT = emp10.loc[emp10.employment == ('Hours_TOT')]
H_TOT.rename(columns = {'value':'H_TOT'}, inplace=True)

H_Emp = emp10.loc[emp10.employment == ('Hours_Emp')]
H_Emp.rename(columns = {'value':'H_EMP'}, inplace=True)

# Split dataframe gva10 in two - GVA and compensation for employees 
GVA = gva10.loc[gva10.variable == ('GVA')]
GVA.rename(columns = {'value':'GVA'}, inplace=True)

ComEmp = gva10.loc[gva10.variable == ('ComEmp')]
ComEmp.rename(columns = {'value':'ComEmp'}, inplace=True)
GVA.head()

,variable,industry,unit,year,GVA
0,GVA,Total,nominal,1966,73555
1,GVA,Total,nominal,1967,80538
2,GVA,Total,nominal,1968,88782
3,GVA,Total,nominal,1969,101289
4,GVA,Total,nominal,1970,112399


In [127]:
# dropper 'employment' and 'variable'
H_TOT.drop(columns=['employment'], inplace = True)
H_Emp.drop(columns=['employment'], inplace = True)
GVA.drop(columns=['variable'], inplace = True)
ComEmp.drop(columns=['variable'], inplace = True)


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [131]:
GVA

,industry,unit,year,GVA
0,Total,nominal,1966,73555
1,Total,nominal,1967,80538
2,Total,nominal,1968,88782
3,Total,nominal,1969,101289
4,Total,nominal,1970,112399
5,Total,nominal,1971,125050
6,Total,nominal,1972,142119
7,Total,nominal,1973,165043
8,Total,nominal,1974,185957
9,Total,nominal,1975,208105


In [145]:
# c. merged data
merged = pd.merge(GVA,ComEmp,how='left',on=['year','industry'])
merged1 = pd.merge(merged,H_TOT,how='left',on=['year','industry'])
df = pd.merge(merged1,H_Emp,how='left',on=['year','industry'])
df.drop(columns=['unit_y'], inplace = True)

df.GVA = dataset.GVA.astype('float')
df.ComEmp = dataset.ComEmp.astype('float')
df

df['Productivity'] = df.GVA / df.H_TOT*1000
df['Hour_wage'] = df.ComEmp / df.H_EMP*1000

#df['P: indexed=2010'] =
#df['H: indexed=2010'] =

#groupby year and industry

# Plot .... 

,industry,unit_x,year,GVA,ComEmp,H_TOT,H_EMP,Productivity,Hour_wage
0,Total,nominal,1966,73555.0,40777.0,4604383.0,3637974.0,15.974996,11.208711
1,Total,nominal,1967,80538.0,45385.0,4537166.0,3605118.0,17.750728,12.589047
2,Total,nominal,1968,88782.0,50493.0,4467413.0,3568827.0,19.873247,14.148346
3,Total,nominal,1969,101289.0,57580.0,4453350.0,3585142.0,22.744451,16.060731
4,Total,nominal,1970,112399.0,65214.0,4449231.0,3608565.0,25.262568,18.072004
5,Total,nominal,1971,125050.0,73729.0,4361538.0,3547451.0,28.671079,20.783656
6,Total,nominal,1972,142119.0,81826.0,4304325.0,3512920.0,33.017721,23.292873
7,Total,nominal,1973,165043.0,93844.0,4248746.0,3475705.0,38.845109,26.999990
8,Total,nominal,1974,185957.0,110260.0,4174298.0,3419815.0,44.548089,32.241510
9,Total,nominal,1975,208105.0,123924.0,3951117.0,3220158.0,52.669916,38.483826


In [ ]:
# skaber nye kolloner - forsøger at erstatte denne kode
emp10['Hours_TOT'] = emp10.value[emp10.employment == 'Hours_TOT']
emp10['Hours_Emp'] = emp10.value[emp10.employment == 'Hours_Emp']
gva10['GVA'] = gva10.value[gva10.variable == 'GVA']
gva10['ComEmp'] = gva10.value[gva10.variable == 'ComEmp']

gva10.tail()

In [84]:
# dropper 'employment'
emp10.drop(columns=['employment'], inplace = True)
emp10.drop(columns=['value'], inplace = True)
gva10.drop(columns=['variable'], inplace = True)
gva10.drop(columns=['value'], inplace = True)

In [94]:
 
#emp10 = emp10.dropna()
#gva10.head()
emp10.tail()
gva10.tail()

,industry,unit,year,GVA,ComEmp


In [86]:
# c. merged data
merged = pd.merge(gva10,emp10,how='left',on=['year','industry'])
merged

,industry,unit,year,GVA,ComEmp,Hours_TOT,Hours_Emp
0,Total,nominal,1966,73555.0,NaN,4604383.0,NaN
1,Total,nominal,1966,73555.0,NaN,NaN,3637974.0
2,Total,nominal,1967,80538.0,NaN,4537166.0,NaN
3,Total,nominal,1967,80538.0,NaN,NaN,3605118.0
4,Total,nominal,1968,88782.0,NaN,4467413.0,NaN
5,Total,nominal,1968,88782.0,NaN,NaN,3568827.0
6,Total,nominal,1969,101289.0,NaN,4453350.0,NaN
7,Total,nominal,1969,101289.0,NaN,NaN,3585142.0
8,Total,nominal,1970,112399.0,NaN,4449231.0,NaN
9,Total,nominal,1970,112399.0,NaN,NaN,3608565.0
